# QuDiPy tutorial: using the single electron Schrödinger equation solver

This tutorial will show how to use the qutils to solve the single electron Schrödinger equation for either 1D or 2D potential landscapes. Make sure you are running this tutorial notebook from the tutorials folder in the qudipy repo.


## 1. First load the relevant modules and initialize SimulationParameters

In [ ]:
# Since we don't actually have a package yet for people to install.. 
# We need to add our folder to the PYTHONPATH in order for import to find qudipy
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import qudipy as qd
import qudipy.potential as pot
import qudipy.qutils as qt
import numpy as np

In [ ]:
# Initialize the constants class with the Si/SiO2 material system 
consts = qd.Constants("Si/SiO2")

## 2a. 1D Schrödinger equation: initialize GridParameters class
Now we will create a GridParameters object with data for a Harmonic oscillator.

In [ ]:
# First define the x-coordinates
x = np.linspace(-70,70,301)*1E-9
# Define harmonic oscillator frequency
omega = 5E12
# Now construct the harmonic potential
harm_pot = 1/2*consts.me*omega**2*np.square(x)
    
# Create a GridParameters object
gparams = pot.GridParameters(x, potential=harm_pot)

## 2b. 1D Schrödinger equation: use qutils to solve schrodinger equation

In [ ]:
# Pass sparams, gparams to the solve_schrodinger_eq qutils method to obtain the eigenvalues and eigenvectors
e_ens, e_vecs = qt.solvers.solve_schrodinger_eq(consts, gparams, n_sols=6)

## 2c. Check results
Our energies should have constant spacing (within some error which can be reduced by increasing the
number of x-coordinate points). The energy spacing should be equal to $\hbar \omega$.

In [ ]:
print(f"Energies = {np.real(e_ens)/consts.e} eV")
print(f"hbar*omega = {consts.hbar*omega/consts.e} eV")
print(f"Calculated energy diffs = {np.diff(np.real(e_ens))/consts.e} eV")

Looks close enough! You can play around with increasing number of x-coordinate points to reduce the error. Now let's plot the first few wavefunctions...

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

fig, axes = plt.subplots(2, 3, figsize=(10, 6))#,
                         #subplot_kw={'xticks': [], 'yticks': []})

fig.subplots_adjust(hspace=0.4, wspace=0.25)

for idx, ax in enumerate(axes.flat):
    ax.plot(gparams.x/1E-9, np.real(e_vecs[idx]))
    ax.set_title("State: "+str(idx))
    ax.set(xlabel='x-coords [nm]')

plt.show()

These look exactly as we expect. So now let's do an example for 2D potentials...
# 3a. 2D Schrödinger equation: initialize GridParameters class
Let's create a new GridParameters object with data for a 2D Harmonic oscillator.

In [ ]:
# First define the x(y)-coordinates
x = np.linspace(-60,60,201)*1E-9
y = np.linspace(-60,60,201)*1E-9
# Define harmonic oscillator frequency
omega = 5E12

# Now initialize the GridParameters class
gparams = pot.GridParameters(x, y)
# Now construct the harmonic potential
harm_pot = 1/2*consts.me*omega**2*(np.square(gparams.x_mesh) + np.square(gparams.y_mesh))

# Update the potential for gparams
gparams.update_potential(harm_pot)

## 2b. 2D Schrödinger equation: use qutils to solve schrodinger equation

In [ ]:
# Pass sparams, gparams to the solve_schrodinger_eq qutils method to obtain the eigenvalues and eigenvectors
e_ens, e_vecs = qt.solvers.solve_schrodinger_eq(consts, gparams, n_sols=6)

## 3c. Check results
We expect the 1st and 2nd excited states to be degenerate since they correspond to the (1,0) and (0,1) 2D harmonic orbital modes (n,m).

In [ ]:
print(f"Energies = {np.real(e_ens)/consts.e} eV")
print(f"hbar*omega = {consts.hbar*omega/consts.e} eV")
print(f"Calculated energy diffs = {np.diff(np.real(e_ens))/consts.e} eV")

Looks good.  Now let's plot the wavefunctions.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 6))#,
                         #subplot_kw={'xticks': [], 'yticks': []})

fig.subplots_adjust(hspace=0.4, wspace=0.25)

for idx, ax in enumerate(axes.flat):
    #ax.imshow(gparams.potential, cmap='viridis', 
    ax.imshow(np.real(np.multiply(e_vecs[idx],e_vecs[idx].conj())), cmap='viridis', 
              extent=[gparams.x.min()/1E-9, gparams.x.max()/1E-9,
                      gparams.y.min()/1E-9, gparams.y.max()/1E-9])
    ax.set_title("State: "+str(idx))
    ax.set(xlabel='x-coords [nm]',ylabel='y-coords [nm]')

plt.show()

We expect the eigenfunctions to be the product of the 1D harmonic orbital states $\phi_{n,m}(x,y) = \phi_n(x)\phi_m(y)$. Of course, not all these states look like that.. But when two eigenfunctions have the same eigenvalue their linear combination is also an eigenfunction. For instance, state 1 is actually a linear combination of the equal energy states $\phi_{0,1}$ and $\phi_{1,0}$
$$\phi_1(x,y) = \alpha\phi_{0,1}(x,y) + \sqrt{1+\alpha^2}\phi_{0,1}(x,y))$$
where $\alpha$ is some constant. So these 2D eigenfunctions are correct!